## PtyPy moonflower example
#### scan model: BlockFull
#### engine: Relaxed Averaged Alternate Projections (RAAR)

In [ ]:
from ptypy.core import Ptycho
from ptypy import utils as u

In [ ]:
# create parameter tree
p = u.Param()

In [ ]:
# set verbose level to interactive
p.verbose_level = "interactive"

In [ ]:
# set home path and io settings (no files saved)
p.io = u.Param()
p.io.rfile = None
p.io.autosave = u.Param(active=False)
p.io.interaction = u.Param(active=False)

In [ ]:
p.io.autoplot = u.Param()
p.io.autoplot.active = True
p.io.autoplot.threaded = False
p.io.autoplot.layout = "jupyter"
p.io.autoplot.interval = 1

In [ ]:
# max 200 frames (128x128px) of diffraction data
p.scans = u.Param()
p.scans.MF = u.Param()
p.scans.MF.name = 'BlockFull'
p.scans.MF.data= u.Param()
p.scans.MF.data.name = 'MoonFlowerScan'
p.scans.MF.data.shape = 128
p.scans.MF.data.num_frames = 200
p.scans.MF.data.save = None
p.scans.MF.data.density = 0.2
p.scans.MF.data.photons = 1e8
p.scans.MF.data.psf = 0.

### Load/prep data and save initial state

In [ ]:
# Load and prepare
P = Ptycho(p,level=4)
# Save initial state
P.copy_state(name="initial")

### Run RAAR with beta = 0.9 and save the final state

In [ ]:
# RAAR reconstrucion engine
engine_pars = u.Param()
engine_pars.name = 'RAAR'
engine_pars.numiter = 100
engine_pars.beta = 0.9
P.run(epars=engine_pars)

In [ ]:
P.copy_state(name="RAAR with beta 0.9")

### Run RAAR with beta = 0.7 and save the final state

In [ ]:
P.restore_state(name="initial")
# Check that we are indeed starting from beginning again
fig = u.plot_client.figure_from_ptycho(P)

In [ ]:
# RAAR reconstrucion engine
engine_pars = u.Param()
engine_pars.name = 'RAAR'
engine_pars.numiter = 100
engine_pars.beta = 0.7
P.run(epars=engine_pars)

In [ ]:
P.copy_state(name="RAAR with beta 0.7")

### Compare reconstructions with different beta

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
fig, axes = plt.subplots(ncols=2, nrows=3, figsize=(4,6))
axes[0,0].set_title("beta = 0.9")
axes[0,0].imshow(np.abs(P.state_dict["RAAR with beta 0.9"]["ob"].storages["SMFG00"].data[0,100:-100,100:-100]))
axes[1,0].imshow(np.angle(P.state_dict["RAAR with beta 0.9"]["ob"].storages["SMFG00"].data[0,100:-100,100:-100]))
ax10 = u.PtyAxis(axes[2,0], channel="c")
ax10.set_data(P.state_dict["RAAR with beta 0.9"]["pr"].storages["SMFG00"].data[0,20:-20,20:-20])
axes[0,1].set_title("beta = 0.7")
axes[0,1].imshow(np.abs(P.state_dict["RAAR with beta 0.7"]["ob"].storages["SMFG00"].data[0,100:-100,100:-100]))
axes[1,1].imshow(np.angle(P.state_dict["RAAR with beta 0.7"]["ob"].storages["SMFG00"].data[0,100:-100,100:-100]))
ax11 = u.PtyAxis(axes[2,1], channel="c")
ax11.set_data(P.state_dict["RAAR with beta 0.7"]["pr"].storages["SMFG00"].data[0,20:-20,20:-20])

In [ ]:
iters_0_7 = np.array([it["iterations"] for it in P.state_dict["RAAR with beta 0.7"]["runtime"]["iter_info"]])
error_0_7 = np.array([it["error"] for it in P.state_dict["RAAR with beta 0.7"]["runtime"]["iter_info"]])

In [ ]:
iters_0_9 = np.array([it["iterations"] for it in P.state_dict["RAAR with beta 0.9"]["runtime"]["iter_info"]])
error_0_9 = np.array([it["error"] for it in P.state_dict["RAAR with beta 0.9"]["runtime"]["iter_info"]])

In [ ]:
plt.figure()
plt.plot(iters_0_7, error_0_7[:,1], label="beta=0.7")
plt.plot(iters_0_9, error_0_9[:,1], label="beta=0.9")
plt.semilogy()
plt.ylabel("Maximum Likelihood error")
plt.xlabel("Nr. of iterations")
plt.legend()
plt.show()